In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import re

from sklearn.preprocessing import MultiLabelBinarizer

### 데이터 불러오기

In [3]:
def JsonToDf(jfile,df):
    with open(jfile, 'rb') as f:
        jf = json.load(f)
    jdf = pd.DataFrame(jf)
    result = pd.concat([df,jdf])
    return result

In [4]:
import json
import os 
#경로 = os.getcwd()
path = '/content/gdrive/MyDrive/세종대학교/2022-2/파이썬기반기계학습/중간고사/data/orig/'
file_names = os.listdir(path)
#print(file_names)

ls11=[]
ls32=[]
df11 = pd.DataFrame()
df32 = pd.DataFrame()

for i in file_names:
    if i[-5:]==".json":
        if i[:2]=="11":
            ls11.append(i)
        elif i[:2]=="32":
            ls32.append(i)

for i in ls11:
    df11 = JsonToDf(path+i,df11)
for i in ls32:
    df32 = JsonToDf(path+i,df32)

df = pd.concat([df11,df32])
df

,application_year,invention_title,ipc_section,register_date,ipc_subclass,ipc_main,register_year,ipc_all,ipc_class,claims,application_date,register_number,documentId,application_number,open_date,open_number,open_year,abstract,applicant_name
0,1979,산 생성능을 개량한 효모변이주를 사용하는 약 · 탁주류의 제조법,C,19801220,C12G,C12G-003/02,1980,C12G-003/02,C12,약·탁주류의 양조에 있어서 산생성능을 개량한 변이주 사키로 마이세스·세레비시에 HM...,19790626,100009041,kr19790002076b1,1019790002076,NaN,NaN,NaN,NaN,NaN
1,1982,코로이드 막걸리 제조법,C,NaN,C12H,C12H-001/04,NaN,C12H-001/04,C12,"막걸리의 원료를 즉, 밀가루, 옥수수가루, 보리 등을 100℃ 이상으로 익혀서 10...",19820203,NaN,kr19820000439a,1019820000439,19831219,1019830009216,1983,NaN,NaN
2,1983,탁주의 제조방법,C,NaN,C12G,C12G-003/00,NaN,C12G-003/00,C12,"탁주 발효후에 후수(後水=割水)를 가하여 규격정도수가 되게하지 않고, 발효과정에서 ...",19830427,NaN,kr19830001790a,1019830001790,19841217,1019840008687,1984,NaN,NaN
3,1983,"탁, 약주의 제조방법",C,NaN,C12G,C12G-003/02,NaN,C12G-003/02,C12,"곡류, 저류 및 발효제, 그리고 물을 주원료로하여 약탁주를 제조하는 공지 방법에 있...",19830523,NaN,kr19830002251a,1019830002251,19841219,1019840008810,1984,NaN,NaN
4,1983,발효법에 의한 사과 및 복숭아 탁주의 제조방법,C,NaN,C12G,C12G-003/02,NaN,C12G-003/02,C12,"본문을 상술한 바와같이 사과, 복숭아를 마쇄하여 기질을 만든것에 공지의 바실루스 델...",19831209,NaN,kr19830005823a,1019830005823,19850711,1019850004265,1985,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,2017,간이금고용 서랍이 구비된 가구용 서랍장,A,20180214,A47B,A47B-067/04,2018,A47B-067/04||A47B-061/00||A47B-088/40||A47B-08...,A47,옷장이나 진열장에 일체로 구비되는 시스템 가구용 서랍장이나 책상용 서랍장 또는 옷 ...,20170516,101831156,kr20170060349b1,1020170060349,NaN,NaN,NaN,본 발명은 간이금고용 서랍이 구비된 가구용 서랍장에 관한 것으로서 종래에서랍장의 서...,NaN
796,2017,서랍별 인출 범위가 임의 조절되는 선택형 전도방지 서랍장,A,20180406,A47B,A47B-088/57,2018,A47B-088/57||A47B-088/40||E05B-065/463,A47,서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1);본체 레...,20170516,101848368,kr20170060498b1,1020170060498,NaN,NaN,NaN,본 발명은 서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1...,NaN
797,2017,서랍의 인출 범위가 임의 조절되는 전도방지형 서랍장,A,20180406,A47B,A47B-088/57,2018,A47B-088/57||A47B-088/40||E05B-065/463,A47,서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1);본체 레...,20170516,101848366,kr20170060504b1,1020170060504,NaN,NaN,NaN,본 발명은 서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1...,NaN
798,2017,이동식 테이블 관리 시스템,A,NaN,A47B,A47B-033/00,NaN,A47B-033/00||A47B-095/00||B08B-001/00||B08B-00...,A47,테이블에 남아 있는 음식물 및 쓰레기를 작동 버튼(101) 또는 원격제어 단말기를 ...,20170519,NaN,kr20170061915a,1020170061915,NaN,1020180126889,NaN,본 발명은 이동식 테이블 관리 시스템에 관한 것으로 테이블에 남아 있는 음식물 및 ...,NaN


In [5]:
import json
import pandas as pd
import os 
#경로 = os.getcwd()
path = '/content/gdrive/MyDrive/세종대학교/2022-2/파이썬기반기계학습/중간고사/data/'
file_names = os.listdir(path)
#print(file_names)

ls11=[]
ls32=[]
df11 = pd.DataFrame()
df32 = pd.DataFrame()

for i in file_names:
    if i[-5:]==".json":
        if i[:2]=="11":
            ls11.append(i)
        elif i[:2]=="32":
            ls32.append(i)

for i in ls11:
    df11 = JsonToDf(path+i,df11)
for i in ls32:
    df32 = JsonToDf(path+i,df32)

ldf = pd.concat([df11,df32])
ldf

,updateDate,LLno,SSno,Stext,SStext,Mtext,Lno,ipc_main,Mno,country_code,Sno,LLtext,documentId,application_number,Ltext,keyword,document_type
0,20220608,C,11111,발효주 제조업,탁주 및 약주 제조업,알코올 음료 제조업,11,C12G-003/02,111,KR,1111,제조업(10~34),kr19790002076b1,1019790002076,음료 제조업,약주제조/탁주제조/효모변이주탁주/양조 /산생성능/주모/방법/술덧/젖산/사과산,B1
1,20220608,C,11111,발효주 제조업,탁주 및 약주 제조업,알코올 음료 제조업,11,C12H-001/04,111,KR,1111,제조업(10~34),kr19820000439a,1019820000439,음료 제조업,곡주제조/막걸리제조/코로이드막걸리/옥수수가루/전류/정전/방법/수소이온/녹말식품/흡습방지,A
2,20220608,C,11111,발효주 제조업,탁주 및 약주 제조업,알코올 음료 제조업,11,C12G-003/00,111,KR,1111,제조업(10~34),kr19830001790a,1019830001790,음료 제조업,곡주제조/탁주제조/탁주제조/발효/규격정도수/희석/방법/밀폐용기/생전분/흑국균,A
3,20220608,C,11111,발효주 제조업,탁주 및 약주 제조업,알코올 음료 제조업,11,C12G-003/02,111,KR,1111,제조업(10~34),kr19830002251a,1019830002251,음료 제조업,약주제조/탁주제조/약주제조/발효제/지하용수/살균정화/방법/무기물/취입/혼합,A
4,20220608,C,11111,발효주 제조업,탁주 및 약주 제조업,알코올 음료 제조업,11,C12G-003/02,111,KR,1111,제조업(10~34),kr19830005823a,1019830005823,음료 제조업,곡주제조/탁주제조/사과및복숭아탁주/마쇄/바실루스델브루엑기/당도/방법/발효/가수분해/살균,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,20220608,C,32099,기타 가구 제조업,그 외 기타 가구 제조업,가구 제조업,32,A47B-067/04,320,KR,3209,제조업(10~34),kr20170060349b1,1020170060349,가구 제조업,플라스틱가구제조/가구제조/서랍장/걸이부/잠금장치/서랍체/장치/서랍장본체/브라켓/레일설치부,B1
796,20220608,C,32099,기타 가구 제조업,그 외 기타 가구 제조업,가구 제조업,32,A47B-088/57,320,KR,3209,제조업(10~34),kr20170060498b1,1020170060498,가구 제조업,플라스틱가구제조/가구제조/서랍장/승강핀/구속핀/전도방지/장치/승강유도레일/절개면/끼움편,B1
797,20220608,C,32099,기타 가구 제조업,그 외 기타 가구 제조업,가구 제조업,32,A47B-088/57,320,KR,3209,제조업(10~34),kr20170060504b1,1020170060504,가구 제조업,플라스틱가구제조/가구제조/서랍장/키조작부/수용부/전도방지/장치/승강유도레일/절개면/끼움편,B1
798,20220608,C,32099,기타 가구 제조업,그 외 기타 가구 제조업,가구 제조업,32,A47B-033/00,320,KR,3209,제조업(10~34),kr20170061915a,1020170061915,가구 제조업,가구제조//시스템/자동스크레퍼/건티슈롤/음식물쓰레기수거함/시스템/작동버튼/세척용롤물...,A


### 라벨 붙이기, 데이터 전처리

In [6]:
input_df = df.fillna('')
label_df = ldf.fillna('')
input_df['text'] = input_df['ipc_main'] + " " + input_df['invention_title'] + " " + input_df['abstract'] + " " + input_df['claims']
input_df['keyword'] = label_df['keyword']
input_df['label'] = label_df['Lno']
input_df

,application_year,invention_title,ipc_section,register_date,ipc_subclass,ipc_main,register_year,ipc_all,ipc_class,claims,...,documentId,application_number,open_date,open_number,open_year,abstract,applicant_name,text,keyword,label
0,1979,산 생성능을 개량한 효모변이주를 사용하는 약 · 탁주류의 제조법,C,19801220,C12G,C12G-003/02,1980,C12G-003/02,C12,약·탁주류의 양조에 있어서 산생성능을 개량한 변이주 사키로 마이세스·세레비시에 HM...,...,kr19790002076b1,1019790002076,,,,,,C12G-003/02 산 생성능을 개량한 효모변이주를 사용하는 약 · 탁주류의 제조...,약주제조/탁주제조/효모변이주탁주/양조 /산생성능/주모/방법/술덧/젖산/사과산,11
1,1982,코로이드 막걸리 제조법,C,,C12H,C12H-001/04,,C12H-001/04,C12,"막걸리의 원료를 즉, 밀가루, 옥수수가루, 보리 등을 100℃ 이상으로 익혀서 10...",...,kr19820000439a,1019820000439,19831219,1019830009216,1983,,,"C12H-001/04 코로이드 막걸리 제조법 막걸리의 원료를 즉, 밀가루, 옥수수...",곡주제조/막걸리제조/코로이드막걸리/옥수수가루/전류/정전/방법/수소이온/녹말식품/흡습방지,11
2,1983,탁주의 제조방법,C,,C12G,C12G-003/00,,C12G-003/00,C12,"탁주 발효후에 후수(後水=割水)를 가하여 규격정도수가 되게하지 않고, 발효과정에서 ...",...,kr19830001790a,1019830001790,19841217,1019840008687,1984,,,C12G-003/00 탁주의 제조방법 탁주 발효후에 후수(後水=割水)를 가하여 규...,곡주제조/탁주제조/탁주제조/발효/규격정도수/희석/방법/밀폐용기/생전분/흑국균,11
3,1983,"탁, 약주의 제조방법",C,,C12G,C12G-003/02,,C12G-003/02,C12,"곡류, 저류 및 발효제, 그리고 물을 주원료로하여 약탁주를 제조하는 공지 방법에 있...",...,kr19830002251a,1019830002251,19841219,1019840008810,1984,,,"C12G-003/02 탁, 약주의 제조방법 곡류, 저류 및 발효제, 그리고 물을 ...",약주제조/탁주제조/약주제조/발효제/지하용수/살균정화/방법/무기물/취입/혼합,11
4,1983,발효법에 의한 사과 및 복숭아 탁주의 제조방법,C,,C12G,C12G-003/02,,C12G-003/02,C12,"본문을 상술한 바와같이 사과, 복숭아를 마쇄하여 기질을 만든것에 공지의 바실루스 델...",...,kr19830005823a,1019830005823,19850711,1019850004265,1985,,,C12G-003/02 발효법에 의한 사과 및 복숭아 탁주의 제조방법 본문을 상술한...,곡주제조/탁주제조/사과및복숭아탁주/마쇄/바실루스델브루엑기/당도/방법/발효/가수분해/살균,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,2017,간이금고용 서랍이 구비된 가구용 서랍장,A,20180214,A47B,A47B-067/04,2018,A47B-067/04||A47B-061/00||A47B-088/40||A47B-08...,A47,옷장이나 진열장에 일체로 구비되는 시스템 가구용 서랍장이나 책상용 서랍장 또는 옷 ...,...,kr20170060349b1,1020170060349,,,,본 발명은 간이금고용 서랍이 구비된 가구용 서랍장에 관한 것으로서 종래에서랍장의 서...,,A47B-067/04 간이금고용 서랍이 구비된 가구용 서랍장 본 발명은 간이금고용 ...,플라스틱가구제조/가구제조/서랍장/걸이부/잠금장치/서랍체/장치/서랍장본체/브라켓/레일설치부,32
796,2017,서랍별 인출 범위가 임의 조절되는 선택형 전도방지 서랍장,A,20180406,A47B,A47B-088/57,2018,A47B-088/57||A47B-088/40||E05B-065/463,A47,서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1);본체 레...,...,kr20170060498b1,1020170060498,,,,본 발명은 서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1...,,A47B-088/57 서랍별 인출 범위가 임의 조절되는 선택형 전도방지 서랍장 본 ...,플라스틱가구제조/가구제조/서랍장/승강핀/구속핀/전도방지/장치/승강유도레일/절개면/끼움편,32
797,2017,서랍의 인출 범위가 임의 조절되는 전도방지형 서랍장,A,20180406,A47B,A47B-088/57,2018,A47B-088/57||A47B-088/40||E05B-065/463,A47,서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1);본체 레...,...,kr20170060504b1,1020170060504,,,,본 발명은 서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1...,,A47B-088/57 서랍의 인출 범위가 임의 조절되는 전도방지형 서랍장 본 발명은...,플라스틱가구제조/가구제조/서랍장/키조작부/수용부/전도방지/장치/승강유도레일/절개면/끼움편,32
798,2017,이동식 테이블 관리 시스템,A,,A47B,A47B-033/00,,A47B-033/00||A47B-095/00||B08B-001/00||B08B-00...,A47,테이블에 남아 있는 음식물 및 쓰레기를 작동 버튼(101) 또는 원격제어 단말기를 ...,...,kr20170061915a,1020170061915,,1020180126889,,본 발명은 이동식 테이블 관리 시스템에 관한 것으로 테이블에 남아 있는 음식물 및 ...,,A47B-033/00 이동식 테이블 관리 시스템 본 발명은 이동식 테이블 관리 시스...,가구제조//시스템/자동스크레퍼/건티슈롤/음식물쓰레기수거함/시스템/작동버튼/세척용롤물...,32


In [7]:
#특수문자 제거
input_df['text'] = input_df['text'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','')

#null값 없는거 확인
input_df.isnull().sum()

#중복데이터 제거
input_df.drop_duplicates(subset=['text'], inplace=True)

#bag of words
uniq = input_df['text'].tolist()
uniq = ''.join(uniq)
uniq = list(set(uniq))
uniq.sort()

print(uniq[0:100])

input_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅀ', 'ㅁ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅎ', 'ㅏ', 'ㅗ', 'ㅛ', 'ㅠ', 'ㅡ', 'ㅣ', '가', '각', '간', '갈', '감', '갑', '값', '갓', '갔', '강', '갖', '같', '개', '객', '갠', '갤', '갬', '갭', '갯', '갱', '걀']


,application_year,invention_title,ipc_section,register_date,ipc_subclass,ipc_main,register_year,ipc_all,ipc_class,claims,...,documentId,application_number,open_date,open_number,open_year,abstract,applicant_name,text,keyword,label
0,1979,산 생성능을 개량한 효모변이주를 사용하는 약 · 탁주류의 제조법,C,19801220,C12G,C12G-003/02,1980,C12G-003/02,C12,약·탁주류의 양조에 있어서 산생성능을 개량한 변이주 사키로 마이세스·세레비시에 HM...,...,kr19790002076b1,1019790002076,,,,,,C12G00302 산 생성능을 개량한 효모변이주를 사용하는 약 탁주류의 제조법 ...,약주제조/탁주제조/효모변이주탁주/양조 /산생성능/주모/방법/술덧/젖산/사과산,11
1,1982,코로이드 막걸리 제조법,C,,C12H,C12H-001/04,,C12H-001/04,C12,"막걸리의 원료를 즉, 밀가루, 옥수수가루, 보리 등을 100℃ 이상으로 익혀서 10...",...,kr19820000439a,1019820000439,19831219,1019830009216,1983,,,C12H00104 코로이드 막걸리 제조법 막걸리의 원료를 즉 밀가루 옥수수가루 보...,곡주제조/막걸리제조/코로이드막걸리/옥수수가루/전류/정전/방법/수소이온/녹말식품/흡습방지,11
2,1983,탁주의 제조방법,C,,C12G,C12G-003/00,,C12G-003/00,C12,"탁주 발효후에 후수(後水=割水)를 가하여 규격정도수가 되게하지 않고, 발효과정에서 ...",...,kr19830001790a,1019830001790,19841217,1019840008687,1984,,,C12G00300 탁주의 제조방법 탁주 발효후에 후수를 가하여 규격정도수가 되게하...,곡주제조/탁주제조/탁주제조/발효/규격정도수/희석/방법/밀폐용기/생전분/흑국균,11
3,1983,"탁, 약주의 제조방법",C,,C12G,C12G-003/02,,C12G-003/02,C12,"곡류, 저류 및 발효제, 그리고 물을 주원료로하여 약탁주를 제조하는 공지 방법에 있...",...,kr19830002251a,1019830002251,19841219,1019840008810,1984,,,C12G00302 탁 약주의 제조방법 곡류 저류 및 발효제 그리고 물을 주원료로하...,약주제조/탁주제조/약주제조/발효제/지하용수/살균정화/방법/무기물/취입/혼합,11
4,1983,발효법에 의한 사과 및 복숭아 탁주의 제조방법,C,,C12G,C12G-003/02,,C12G-003/02,C12,"본문을 상술한 바와같이 사과, 복숭아를 마쇄하여 기질을 만든것에 공지의 바실루스 델...",...,kr19830005823a,1019830005823,19850711,1019850004265,1985,,,C12G00302 발효법에 의한 사과 및 복숭아 탁주의 제조방법 본문을 상술한 바...,곡주제조/탁주제조/사과및복숭아탁주/마쇄/바실루스델브루엑기/당도/방법/발효/가수분해/살균,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,2017,간이금고용 서랍이 구비된 가구용 서랍장,A,20180214,A47B,A47B-067/04,2018,A47B-067/04||A47B-061/00||A47B-088/40||A47B-08...,A47,옷장이나 진열장에 일체로 구비되는 시스템 가구용 서랍장이나 책상용 서랍장 또는 옷 ...,...,kr20170060349b1,1020170060349,,,,본 발명은 간이금고용 서랍이 구비된 가구용 서랍장에 관한 것으로서 종래에서랍장의 서...,,A47B06704 간이금고용 서랍이 구비된 가구용 서랍장 본 발명은 간이금고용 서랍...,플라스틱가구제조/가구제조/서랍장/걸이부/잠금장치/서랍체/장치/서랍장본체/브라켓/레일설치부,32
796,2017,서랍별 인출 범위가 임의 조절되는 선택형 전도방지 서랍장,A,20180406,A47B,A47B-088/57,2018,A47B-088/57||A47B-088/40||E05B-065/463,A47,서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1);본체 레...,...,kr20170060498b1,1020170060498,,,,본 발명은 서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1...,,A47B08857 서랍별 인출 범위가 임의 조절되는 선택형 전도방지 서랍장 본 발명...,플라스틱가구제조/가구제조/서랍장/승강핀/구속핀/전도방지/장치/승강유도레일/절개면/끼움편,32
797,2017,서랍의 인출 범위가 임의 조절되는 전도방지형 서랍장,A,20180406,A47B,A47B-088/57,2018,A47B-088/57||A47B-088/40||E05B-065/463,A47,서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1);본체 레...,...,kr20170060504b1,1020170060504,,,,본 발명은 서랍에 대응하는 개소로 층간 구분하여 레일(2)이 고정 배열되는 본체(1...,,A47B08857 서랍의 인출 범위가 임의 조절되는 전도방지형 서랍장 본 발명은 서...,플라스틱가구제조/가구제조/서랍장/키조작부/수용부/전도방지/장치/승강유도레일/절개면/끼움편,32
798,2017,이동식 테이블 관리 시스템,A,,A47B,A47B-033/00,,A47B-033/00||A47B-095/00||B08B-001/00||B08B-00...,A47,테이블에 남아 있는 음식물 및 쓰레기를 작동 버튼(101) 또는 원격제어 단말기를 ...,...,kr20170061915a,1020170061915,,1020180126889,,본 발명은 이동식 테이블 관리 시스템에 관한 것으로 테이블에 남아 있는 음식물 및 ...,,A47B03300 이동식 테이블 관리 시스템 본 발명은 이동식 테이블 관리 시스템에...,가구제조//시스템/자동스크레퍼/건티슈롤/음식물쓰레기수거함/시스템/작동버튼/세척용롤물...,32


In [8]:
#Tokenizer
#가->1 ,나->2

from tensorflow.keras.preprocessing.text import Tokenizer

#글자 단위 정수로 변환 , Flase는 단어단위, <OOV>는 없는 단어
tokenizer = Tokenizer(char_level=True, oov_token='<OOV>')

text_list = input_df['text'].tolist()
tokenizer.fit_on_texts(text_list)

print(tokenizer.word_index)
print(text_list[0:10])

train_seq = tokenizer.texts_to_sequences(text_list)
print(train_seq[0:10])

Y = input_df['label'].tolist()
print(Y[0:10])

{'<OOV>': 1, ' ': 2, '하': 3, '는': 4, '이': 5, '에': 6, '상': 7, '기': 8, '의': 9, '을': 10, '부': 11, '0': 12, '로': 13, '1': 14, '수': 15, '되': 16, '를': 17, '제': 18, '가': 19, '지': 20, '2': 21, '고': 22, '성': 23, '한': 24, '으': 25, '어': 26, '조': 27, '서': 28, '구': 29, '용': 30, '단': 31, '리': 32, '된': 33, '여': 34, '과': 35, '도': 36, '다': 37, '3': 38, '시': 39, '형': 40, '정': 41, '치': 42, '장': 43, '와': 44, '있': 45, '면': 46, '방': 47, '전': 48, '스': 49, '결': 50, '공': 51, '자': 52, '발': 53, '대': 54, '4': 55, '함': 56, '및': 57, '물': 58, '음': 59, '판': 60, '합': 61, '은': 62, '동': 63, '사': 64, '체': 65, '트': 66, '5': 67, '것': 68, '재': 69, '측': 70, '내': 71, '레': 72, '포': 73, '분': 74, '일': 75, '계': 76, '게': 77, '해': 78, '출': 79, '록': 80, '각': 81, '본': 82, '프': 83, '연': 84, '관': 85, '유': 86, '효': 87, '비': 88, '며': 89, '주': 90, '간': 91, '중': 92, '위': 93, '침': 94, '소': 95, '나': 96, 'a': 97, '매': 98, '통': 99, '개': 100, '인': 101, '설': 102, '능': 103, '입': 104, '받': 105, '착': 106, '7': 107, '안': 108, '라': 109, '특': 110, '징'

In [15]:
#문장 길이 제한
input_df['length'] = input_df['text'].str.len()

print(input_df.head())
print(input_df.describe())

#2000자 이하 개수 세기
print(input_df['length'][input_df['length']<2000].count())

#글자수 제한
from tensorflow.keras.preprocessing.sequence import pad_sequences
X=pad_sequences(train_seq, maxlen=2000)

  application_year                      invention_title ipc_section  \
0             1979  산 생성능을 개량한 효모변이주를 사용하는 약 · 탁주류의 제조법           C   
1             1982                         코로이드 막걸리 제조법           C   
2             1983                             탁주의 제조방법           C   
3             1983                          탁, 약주의 제조방법           C   
4             1983            발효법에 의한 사과 및 복숭아 탁주의 제조방법           C   

  register_date ipc_subclass     ipc_main register_year      ipc_all  \
0      19801220         C12G  C12G-003/02          1980  C12G-003/02   
1                       C12H  C12H-001/04                C12H-001/04   
2                       C12G  C12G-003/00                C12G-003/00   
3                       C12G  C12G-003/02                C12G-003/02   
4                       C12G  C12G-003/02                C12G-003/02   

  ipc_class                                             claims  ...  \
0       C12  약·탁주류의 양조에 있어서 산생성능을 개량한 변이주 사키로 마이세스·세레비시에 HM...  ...  

#학습/테스트/검증 데이터 분리

In [10]:
#train/val 데이터 나누기
from sklearn.model_selection import train_test_split
trainX, valX, trainY, valY = train_test_split(X,Y,test_size=0.2, random_state=42)

print(len(trainX))
print(len(valX))

6452
1613


### 훈련셋, 테스트셋 .csv 파일로 출력

In [11]:
trainX=pd.DataFrame(trainX)
trainY=pd.DataFrame(trainY)
valX=pd.DataFrame(valX)
valY=pd.DataFrame(valY)

trainX.to_csv("x_train.csv",encoding='utf-8-sig',index=False)
trainY.to_csv("y_train.csv",encoding='utf-8-sig',index=False)
valX.to_csv("x_test.csv",encoding='utf-8-sig',index=False)
valY.to_csv("y_test.csv",encoding='utf-8-sig',index=False)

### Xgboost 모델 테스트

In [12]:
x = pd.read_csv("/content/x_train.csv")
y = pd.read_csv("/content/y_train.csv")
x_test = pd.read_csv("/content/x_test.csv")
y_test = pd.read_csv("/content/y_test.csv")
x

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0,0,0,0,0,0,0,0,0,0,...,2,137,144,15,2,51,155,2,43,42
1,85,56,35,7,8,2,117,59,123,85,...,17,2,29,88,3,4,2,41,15,8
2,0,0,0,0,0,0,0,0,0,0,...,2,117,59,30,8,2,18,27,43,42
3,0,0,0,0,0,0,0,0,0,0,...,2,189,202,90,9,2,18,27,47,124
4,0,0,0,0,0,0,0,0,0,0,...,94,54,2,98,66,32,49,2,5,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6447,0,0,0,0,0,0,0,0,0,0,...,406,5,17,2,31,2,487,488,2,118
6448,0,0,0,0,0,0,0,0,0,0,...,7,60,2,48,46,2,166,167,11,69
6449,0,0,0,0,0,0,0,0,0,0,...,24,2,53,87,90,2,18,27,47,124
6450,0,0,0,0,0,0,0,0,0,0,...,4,2,156,5,162,40,23,2,9,52


In [16]:
import xgboost
model = xgboost.XGBClassifier(n_estimators=200)

model.fit(x,y)
pred = model.predict(x_test)
print(pred)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[11 11 32 ... 32 11 32]


In [17]:
from sklearn.metrics import accuracy_score
 # 평가하기
accuracy = accuracy_score(y_test, pred)
print('Accuracy : %.2f%%'%(accuracy*100))

Accuracy : 92.00%


</br></br></br></br></br>